<a href="https://colab.research.google.com/github/TanmayBhatt45/Resume-scoring/blob/main/Resume_Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install PyPDF2
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.3 MB/s eta 0:00:00


In [3]:
!pip uninstall -y nltk
!pip install --no-cache-dir nltk

Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.9 MB/s eta 0:00:00


In [4]:
!pip install --upgrade --no-cache-dir nltk
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
!pip install textstat
!pip install language-tool-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.7 MB/s eta 0:00:00


In [6]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [7]:
!pip uninstall -y spacy
!pip install --no-cache-dir spacy
!python -m spacy download en_core_web_md
!pip install --no-cache-dir torch torchvision torchaudio

Found existing installation: spacy 3.8.4
Uninstalling spacy-3.8.4:
  Successfully uninstalled spacy-3.8.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 51.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 143.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 206.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 207.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 219.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 183.0 MB/s eta 

In [56]:
import PyPDF2
import docx
import re
import os
import nltk
import textstat
import language_tool_python
import spacy
from multiprocessing import Pool
from sklearn.feature_extraction.text import CountVectorizer

# Ensure necessary NLTK resources are downloaded
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

tool = language_tool_python.LanguageTool('en-US')  # Grammar checker
nlp = spacy.load("en_core_web_md")

def preprocess_text(text):
    return " ".join([word for word in text.split() if word.isalpha()])

def detect_job_industry(job_description):
    industry_keywords = {
        "software": ["developer", "engineer", "programmer", "AI", "ML", "data scientist"],
        "sales": ["sales", "marketing", "business development"],
        "security": ["security guard", "patrol", "law enforcement", "CCTV", "fire safety"],
        "finance": ["finance", "accounting", "investment"],
        "customer support": ["customer service", "support", "helpdesk"]
    }

    for industry, keywords in industry_keywords.items():
        if any(term in job_description.lower() for term in keywords):
            return industry
    return "general"

class CVAnalyzer:
    def __init__(self, cv_path):
        self.cv_path = cv_path
        self.text = preprocess_text(self.extract_text().lower())

    def extract_text(self):
        if self.cv_path.endswith('.pdf'):
            pdf_reader = PyPDF2.PdfReader(self.cv_path)
            text = "".join([page.extract_text() for page in pdf_reader.pages if page.extract_text()])
        elif self.cv_path.endswith('.docx'):
            doc = docx.Document(self.cv_path)
            text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
        else:
            raise ValueError("Unsupported file format")
        return text

    def analyze_language_quality(self):
        readability = textstat.flesch_reading_ease(self.text)
        grammar_errors = len(tool.check(self.text))
        grammar_score = max(0, 1 - (grammar_errors / 50)) * 100
        return max((readability * 0.5) + (grammar_score * 0.5), 10)

    def match_resume_industry(self, job_description):
      job_industry = detect_job_industry(job_description)
      resume_industry = detect_job_industry(self.text)

      if job_industry == resume_industry:
        return 1.0  # Perfect match ✅
      elif job_industry == "general" or resume_industry == "general":
        return 0.4  # Partial match ⚠️
      else:
        return 0.1  # Mismatch penalty ❌


    def extract_experience(self, job_description):
        experience_years = re.findall(r'(\d+(?:\.\d+)?)\s*(?:year|years|yr|yrs|months|mo)', self.text, re.IGNORECASE)
        total_experience = sum(map(float, experience_years)) if experience_years else 0

        role_match = self.match_resume_industry(job_description)
        return max((total_experience * 5 if role_match else total_experience * 1.5), 5)

    def extract_location(self, city, country, job_description):
        city_match = city.lower() in self.text
        country_match = country.lower() in self.text
        location_weight = 50 if "remote" in job_description.lower() else 100
        if city_match and country_match:
            return 1.2 * location_weight
        elif country_match:
            return 0.3 * location_weight
        else:
            return 0.05 * location_weight

    def match_job_keywords(self, job_description):
      doc1 = nlp(self.text)
      doc2 = nlp(job_description)
      embedding_score = doc1.similarity(doc2) * 100 if doc1.has_vector and doc2.has_vector else 0
      keyword_score = self.keyword_match(job_description)

      role_match = self.match_resume_industry(job_description)
      final_score = ((embedding_score * 0.6) + (keyword_score * 0.4)) * role_match

      final_score = min(final_score, 100)  # 🔥 Cap the job match score at 100

      if role_match < 0.5:
        final_score = min(final_score, 25)  # ❌ Cap at 25 for bad matches

      return final_score



    def keyword_match(self, job_text):
        vectorizer = CountVectorizer(stop_words='english')
        vectors = vectorizer.fit_transform([self.text, job_text])
        return (vectors * vectors.T).toarray()[0, 1] * 100

def score_candidate(cv_path, weights, city, country, job_description):
    analyzer = CVAnalyzer(cv_path)

    role_match_multiplier = analyzer.match_resume_industry(job_description)

    language_quality_score = analyzer.analyze_language_quality() * role_match_multiplier
    experience_quality_score = analyzer.extract_experience(job_description) * role_match_multiplier
    years_of_experience_score = analyzer.extract_experience(job_description) * role_match_multiplier
    location_score = analyzer.extract_location(city, country, job_description)
    job_match_score = analyzer.match_job_keywords(job_description)

    print(f"DEBUG: {cv_path}\n Language Quality: {language_quality_score}\n Experience Quality: {experience_quality_score}\n Years of Experience: {years_of_experience_score}\n Location Score: {location_score}\n Job Match Score: {job_match_score}")

    overall_score = (
        (weights['language_quality'] * language_quality_score) +
        (weights['experience_quality'] * experience_quality_score * 1.5) +
        (weights['years_of_experience'] * years_of_experience_score *1.5) +
        (weights['location'] * location_score * 2) +
        (weights['job_match'] * job_match_score * 3)
    )

    return min(overall_score, 100)

def batch_process(cv_list, weights, city, country, job_description):
    with Pool(processes=os.cpu_count()) as pool:
        scores = pool.starmap(score_candidate, [(cv, weights, city, country, job_description) for cv in cv_list])
    return scores

# Example usage
weights = {'language_quality': 0.2, 'experience_quality': 0.2, 'years_of_experience': 0.2, 'location': 0.2, 'job_match': 0.2}
city = ''# Enter city of recruitment
country = '' # Enter city of recruitment
job_description = "" # Enter Job Description
cv_files = ["Example1.pdf", "example2.docx"]

scores = batch_process(cv_files, weights, city, country, job_description)
print("Candidate Scores:", scores)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


DEBUG: /content/Resume Tanmay.pdf
 Language Quality: 10.0
 Experience Quality: 5.0
 Years of Experience: 5.0
 Location Score: 60.0
 Job Match Score: 100
DEBUG: /content/Resume_Security_Guard.pdf
 Language Quality: 2.9825
 Experience Quality: 0.5
 Years of Experience: 0.5
 Location Score: 15.0
 Job Match Score: 5.0877445936203
DEBUG: /content/Resume_Customer_Support.pdf
 Language Quality: 3.507
 Experience Quality: 0.5
 Years of Experience: 0.5
 Location Score: 15.0
 Job Match Score: 5.149150729179382
DEBUG: /content/Resume_Financial_Analyst.pdf
 Language Quality: 2.8115
 Experience Quality: 0.5
 Years of Experience: 0.5
 Location Score: 15.0
 Job Match Score: 4.823382496833801
DEBUG: /content/Resume Tanuj Dwivedi.pdf
 Language Quality: 10.0
 Experience Quality: 5.0
 Years of Experience: 5.0
 Location Score: 15.0
 Job Match Score: 100
DEBUG: /content/Resume_Sales_Manager.pdf
 Language Quality: 4.404
 Experience Quality: 0.5
 Years of Experience: 0.5
 Location Score: 15.0
 Job Match Scor